In [1]:
import numpy as np
import pandas as pd
import os 
import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import numba
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [ ]:
path  = './data/'

train_sales  = pd.read_csv(path+'train_sales_data.csv')    #历史销量数据
train_search = pd.read_csv(path+'train_search_data.csv')    #车型搜索数据
train_user   = pd.read_csv(path+'train_user_reply_data.csv')#汽车垂直媒体新闻评论数据和车型评论数据

evaluation_public = pd.read_csv(path+'evaluation_public.csv')
submit_example    = pd.read_csv(path+'submit_example.csv')


#滞后特征
lag = train_sales.salesVolume.values

pad_4 = np.zeros(1320*4, dtype = int)

lag_4 = np.append(pad_4, lag)

pad_5 = np.zeros(1320*5, dtype = int)

lag_5 = np.append(pad_5, lag)

lag_5 = lag_5[:-1320]


#小值进行替代，comment小于50的值换为50，news小于300的值换为300

list_comment = train_user[train_user.carCommentVolum<50].index
for i in list_comment:
    train_user.iloc[i,-2] = 50
    
list_news = train_user[train_user.newsReplyVolum<300].index
for i in list_news:
    train_user.iloc[i,-1] = 300

car_model = train_search.model.unique()
province = train_search.province.unique()


#popularity特征
#2018年1月，提取方式历史月份销量比例，考虑时间衰减，月份越近占比越高
for model in car_model:
    for ss in province:
        m1_12    = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==12)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values
        m1_11    = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==11)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values
        m1_10    = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==10)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values
        m1_09    = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==9)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values
        m1_08    = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==8)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values
        
        m1_12_volum = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==12)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values * m1_12
        m1_11_volum = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==11)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values * m1_11
        m1_10_volum = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==10)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values * m1_10
        m1_09_volum = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==9)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values * m1_09
        m1_08_volum = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==8)&(train_search.model==model)&(train_search.province==ss), 'popularity'].values * m1_08


        evaluation_public.loc[(evaluation_public.regMonth==1)&(evaluation_public.model==model)&(evaluation_public.province==ss), 'popularity'] = np.array(m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/16 + m1_08_volum/16)

#某一车型在这一月的搜索量 用这一车型2017年，及它对于2016年比例来定
for model in car_model:
    for ss in province:
        m16_1_2  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_1_3  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_1_4  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==4)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_1_5  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==5)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values

        m16_2_3  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_2_4  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==4)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_2_5  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==5)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_2_6  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==6)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values

        m16_3_4  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==4)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_3_5  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==5)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_3_6  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==6)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m16_3_7  = train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2016)&(train_search.regMonth==7)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values

        m17_1_2  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_1_3  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_1_4  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==4)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_1_5  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==5)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values


        m17_2_3  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_2_4  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==4)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_2_5  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==5)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_2_6  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==6)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values

        m17_3_4  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==4)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_3_5  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==5)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_3_6  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==6)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values
        m17_3_7  = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==7)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values

        m16_1 = m16_1_2/2 + m16_1_3/4 + m16_1_4/8 + m16_1_5/8
        m16_2 = m16_2_3/2 + m16_2_4/4 + m16_2_5/8 + m16_2_6/8
        m16_3 = m16_3_4/2 + m16_3_5/4 + m16_3_6/8 + m16_3_7/8

        m17_1 = m17_1_2/2 + m17_1_3/4 + m17_1_4/8 + m17_1_5/8
        m17_2 = m17_2_3/2 + m17_2_4/4 + m17_2_5/8 + m17_2_6/8
        m17_3 = m17_3_4/2 + m17_3_5/4 + m17_3_6/8 + m17_3_7/8

        m1 = m16_1 * 0.4 +  m17_1 * 0.6
        m2 = m16_2 * 0.4 +  m17_2 * 0.6
        m3 = m16_3 * 0.4 +  m17_3 * 0.6

        evaluation_public.loc[(evaluation_public.regMonth==2)&(evaluation_public.model==model)&(evaluation_public.province==ss), 'popularity'] = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==1)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / m1
        evaluation_public.loc[(evaluation_public.regMonth==3)&(evaluation_public.model==model)&(evaluation_public.province==ss), 'popularity'] = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==2)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / m2
        evaluation_public.loc[(evaluation_public.regMonth==4)&(evaluation_public.model==model)&(evaluation_public.province==ss), 'popularity'] = train_search.loc[(train_search.regYear==2017)&(train_search.regMonth==3)&(train_search.model==model)&(train_search.province==ss) , 'popularity'].values / m3

#carcommentvolum特征
#同一model 月的数据
#2018年1月，提取方式历史月份销量比例，考虑时间衰减，月份越近占比越高
for model in car_model:
    m1_12    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==12)&(train_user.model==model), 'carCommentVolum'].values
    m1_11    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==11)&(train_user.model==model), 'carCommentVolum'].values
    m1_10    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==10)&(train_user.model==model), 'carCommentVolum'].values
    m1_09    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==9)&(train_user.model==model) , 'carCommentVolum'].values
    m1_08    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==8)&(train_user.model==model) , 'carCommentVolum'].values


    m1_12_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==12)&(train_user.model==model), 'carCommentVolum'].values * m1_12
    m1_11_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==11)&(train_user.model==model), 'carCommentVolum'].values * m1_11
    m1_10_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==10)&(train_user.model==model), 'carCommentVolum'].values * m1_10
    m1_09_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==9)&(train_user.model==model) , 'carCommentVolum'].values * m1_09
    m1_08_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==8)&(train_user.model==model) , 'carCommentVolum'].values * m1_08


    evaluation_public.loc[(evaluation_public.regMonth==1)&(evaluation_public.model==model), 'carCommentVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/16 + m1_08_volum/16


for model in car_model:
    m16_1_2  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values
    m16_1_3  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values
    m16_1_4  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==4)&(train_user.model==model) , 'carCommentVolum'].values
    m16_1_5  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==5)&(train_user.model==model) , 'carCommentVolum'].values

    m16_2_3  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values
    m16_2_4  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==4)&(train_user.model==model) , 'carCommentVolum'].values
    m16_2_5  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==5)&(train_user.model==model) , 'carCommentVolum'].values
    m16_2_6  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==6)&(train_user.model==model) , 'carCommentVolum'].values

    m16_3_4  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==4)&(train_user.model==model) , 'carCommentVolum'].values
    m16_3_5  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==5)&(train_user.model==model) , 'carCommentVolum'].values
    m16_3_6  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==6)&(train_user.model==model) , 'carCommentVolum'].values
    m16_3_7  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==7)&(train_user.model==model) , 'carCommentVolum'].values

    m17_1_2  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values
    m17_1_3  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values
    m17_1_4  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==4)&(train_user.model==model) , 'carCommentVolum'].values
    m17_1_5  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==5)&(train_user.model==model) , 'carCommentVolum'].values


    m17_2_3  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values
    m17_2_4  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==4)&(train_user.model==model) , 'carCommentVolum'].values
    m17_2_5  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==5)&(train_user.model==model) , 'carCommentVolum'].values
    m17_2_6  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==6)&(train_user.model==model) , 'carCommentVolum'].values

    m17_3_4  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==4)&(train_user.model==model) , 'carCommentVolum'].values
    m17_3_5  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==5)&(train_user.model==model) , 'carCommentVolum'].values
    m17_3_6  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==6)&(train_user.model==model) , 'carCommentVolum'].values
    m17_3_7  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==7)&(train_user.model==model) , 'carCommentVolum'].values

    m16_1 = m16_1_2/2 + m16_1_3/4 + m16_1_4/8 + m16_1_5/8
    m16_2 = m16_2_3/2 + m16_2_4/4 + m16_2_5/8 + m16_2_6/8
    m16_3 = m16_3_4/2 + m16_3_5/4 + m16_3_6/8 + m16_3_7/8

    m17_1 = m17_1_2/2 + m17_1_3/4 + m17_1_4/8 + m17_1_5/8
    m17_2 = m17_2_3/2 + m17_2_4/4 + m17_2_5/8 + m17_2_6/8
    m17_3 = m17_3_4/2 + m17_3_5/4 + m17_3_6/8 + m17_3_7/8

    m1 = m16_1 * 0.4 +  m17_1 * 0.6
    m2 = m16_2 * 0.4 +  m17_2 * 0.6
    m3 = m16_3 * 0.4 +  m17_3 * 0.6

    evaluation_public.loc[(evaluation_public.regMonth==2)&(evaluation_public.model==model), 'carCommentVolum'] = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'carCommentVolum'].values / m1
    evaluation_public.loc[(evaluation_public.regMonth==3)&(evaluation_public.model==model), 'carCommentVolum'] = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'carCommentVolum'].values / m2
    evaluation_public.loc[(evaluation_public.regMonth==4)&(evaluation_public.model==model), 'carCommentVolum'] = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'carCommentVolum'].values / m3

#newsReplyVolum特征
#同一model 月的数据
#2018年1月，提取方式历史月份销量比例，考虑时间衰减，月份越近占比越高
for model in car_model:
    m1_12    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==12)&(train_user.model==model), 'newsReplyVolum'].values
    m1_11    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==11)&(train_user.model==model), 'newsReplyVolum'].values
    m1_10    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==10)&(train_user.model==model), 'newsReplyVolum'].values
    m1_09    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==9)&(train_user.model==model) , 'newsReplyVolum'].values
    m1_08    = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==8)&(train_user.model==model) , 'newsReplyVolum'].values


    m1_12_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==12)&(train_user.model==model), 'newsReplyVolum'].values * m1_12
    m1_11_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==11)&(train_user.model==model), 'newsReplyVolum'].values * m1_11
    m1_10_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==10)&(train_user.model==model), 'newsReplyVolum'].values * m1_10
    m1_09_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==9)&(train_user.model==model) , 'newsReplyVolum'].values * m1_09
    m1_08_volum = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==8)&(train_user.model==model) , 'newsReplyVolum'].values * m1_08


    evaluation_public.loc[(evaluation_public.regMonth==1)&(evaluation_public.model==model), 'newsReplyVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/16 + m1_08_volum/16


for model in car_model:
    m16_1_2  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_1_3  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_1_4  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==4)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_1_5  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==5)&(train_user.model==model) , 'newsReplyVolum'].values

    m16_2_3  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_2_4  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==4)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_2_5  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==5)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_2_6  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==6)&(train_user.model==model) , 'newsReplyVolum'].values

    m16_3_4  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==4)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_3_5  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==5)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_3_6  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==6)&(train_user.model==model) , 'newsReplyVolum'].values
    m16_3_7  = train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2016)&(train_user.regMonth==7)&(train_user.model==model) , 'newsReplyVolum'].values

    m17_1_2  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_1_3  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_1_4  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==4)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_1_5  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==5)&(train_user.model==model) , 'newsReplyVolum'].values


    m17_2_3  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_2_4  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==4)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_2_5  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==5)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_2_6  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==6)&(train_user.model==model) , 'newsReplyVolum'].values

    m17_3_4  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==4)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_3_5  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==5)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_3_6  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==6)&(train_user.model==model) , 'newsReplyVolum'].values
    m17_3_7  = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==7)&(train_user.model==model) , 'newsReplyVolum'].values

    m16_1 = m16_1_2/2 + m16_1_3/4 + m16_1_4/8 + m16_1_5/8
    m16_2 = m16_2_3/2 + m16_2_4/4 + m16_2_5/8 + m16_2_6/8
    m16_3 = m16_3_4/2 + m16_3_5/4 + m16_3_6/8 + m16_3_7/8

    m17_1 = m17_1_2/2 + m17_1_3/4 + m17_1_4/8 + m17_1_5/8
    m17_2 = m17_2_3/2 + m17_2_4/4 + m17_2_5/8 + m17_2_6/8
    m17_3 = m17_3_4/2 + m17_3_5/4 + m17_3_6/8 + m17_3_7/8

    m1 = m16_1 * 0.4 +  m17_1 * 0.6
    m2 = m16_2 * 0.4 +  m17_2 * 0.6
    m3 = m16_3 * 0.4 +  m17_3 * 0.6

    evaluation_public.loc[(evaluation_public.regMonth==2)&(evaluation_public.model==model), 'newsReplyVolum'] = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==1)&(train_user.model==model) , 'newsReplyVolum'].values / m1
    evaluation_public.loc[(evaluation_public.regMonth==3)&(evaluation_public.model==model), 'newsReplyVolum'] = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==2)&(train_user.model==model) , 'newsReplyVolum'].values / m2
    evaluation_public.loc[(evaluation_public.regMonth==4)&(evaluation_public.model==model), 'newsReplyVolum'] = train_user.loc[(train_user.regYear==2017)&(train_user.regMonth==3)&(train_user.model==model) , 'newsReplyVolum'].values / m3

#forecastVolum

#预测值的特征设计2016,2017年也采用这个方法。
#所在月份前5个月的数据*（历史该月份/历史该月份相对历史该月份前5个月的比值）
#2018年1月，提取方式历史月份销量比例，考虑时间衰减，月份越近占比越高
month = [1,2,3,4,5,6,7,8,9,10,11,12]
#2017 5-12月
for i in month[4:]:
    m1_12_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-1)), 'salesVolume'].values 
    m1_11_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-2)), 'salesVolume'].values 
    m1_10_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-3)), 'salesVolume'].values 
    m1_09_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-4)) , 'salesVolume'].values


    train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==i), 'forecastVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/8

#2017 1月
i = 1
m1_12_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-1+12)), 'salesVolume'].values
m1_11_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-2+12)), 'salesVolume'].values 
m1_10_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-3+12)), 'salesVolume'].values 
m1_09_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-4+12)) , 'salesVolume'].values 


train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==i), 'forecastVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/8

#2017 2月
i = 2
m1_12_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-1)), 'salesVolume'].values
m1_11_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-2+12)), 'salesVolume'].values
m1_10_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-3+12)), 'salesVolume'].values
m1_09_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-4+12)) , 'salesVolume'].values


train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==i), 'forecastVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/8

#2017 3月
i = 3
m1_12_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-1)), 'salesVolume'].values
m1_11_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-2)), 'salesVolume'].values
m1_10_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-3+12)), 'salesVolume'].values
m1_09_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-4+12)) , 'salesVolume'].values


train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==i), 'forecastVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/8

#2017 4月
i = 4
m1_12_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-1)), 'salesVolume'].values 
m1_11_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-2)), 'salesVolume'].values 
m1_10_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==(i-3)), 'salesVolume'].values
m1_09_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-4+12)) , 'salesVolume'].values


train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==i), 'forecastVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/8



#2016年5-12月
for i in month[4:]:
    m1_12_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-1)), 'salesVolume'].values
    m1_11_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-2)), 'salesVolume'].values
    m1_10_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-3)), 'salesVolume'].values
    m1_09_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==(i-4)) , 'salesVolume'].values


    train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==i), 'forecastVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/8

#forecastVolum
#2018年1月，提取方式历史月份销量比例，考虑时间衰减，月份越近占比越高
m1_12    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12), 'salesVolume'].values
m1_11    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11), 'salesVolume'].values
m1_10    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10), 'salesVolume'].values
m1_09    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values
m1_08    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==8) , 'salesVolume'].values


m1_12_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==12), 'salesVolume'].values * m1_12
m1_11_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==11), 'salesVolume'].values * m1_11
m1_10_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==10), 'salesVolume'].values * m1_10
m1_09_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==9) , 'salesVolume'].values * m1_09
m1_08_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==8) , 'salesVolume'].values * m1_08


evaluation_public.loc[evaluation_public.regMonth==1, 'forecastVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/16 + m1_08_volum/16


#forecastVolum
m16_1_2  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values
m16_1_3  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values
m16_1_4  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values
m16_1_5  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values

m16_2_3  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values
m16_2_4  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values
m16_2_5  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values
m16_2_6  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==6) , 'salesVolume'].values

m16_3_4  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values
m16_3_5  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values
m16_3_6  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==6) , 'salesVolume'].values
m16_3_7  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==7) , 'salesVolume'].values

m17_1_2  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values
m17_1_3  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values
m17_1_4  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m17_1_5  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values


m17_2_3  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values
m17_2_4  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m17_2_5  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values
m17_2_6  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==6) , 'salesVolume'].values

m17_3_4  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m17_3_5  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values
m17_3_6  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==6) , 'salesVolume'].values
m17_3_7  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==7) , 'salesVolume'].values

m16_1 = m16_1_2/2 + m16_1_3/4 + m16_1_4/8 + m16_1_5/8
m16_2 = m16_2_3/2 + m16_2_4/4 + m16_2_5/8 + m16_2_6/8
m16_3 = m16_3_4/2 + m16_3_5/4 + m16_3_6/8 + m16_3_7/8

m17_1 = m17_1_2/2 + m17_1_3/4 + m17_1_4/8 + m17_1_5/8
m17_2 = m17_2_3/2 + m17_2_4/4 + m17_2_5/8 + m17_2_6/8
m17_3 = m17_3_4/2 + m17_3_5/4 + m17_3_6/8 + m17_3_7/8

m1 = m16_1 * 0.4 +  m17_1 * 0.6
m2 = m16_2 * 0.4 +  m17_2 * 0.6
m3 = m16_3 * 0.4 +  m17_3 * 0.6

evaluation_public.loc[evaluation_public.regMonth==2, 'forecastVolum'] = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / m1
evaluation_public.loc[evaluation_public.regMonth==3, 'forecastVolum'] = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / m2
evaluation_public.loc[evaluation_public.regMonth==4, 'forecastVolum'] = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / m3

#合并
data = train_sales.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth'])

data = pd.concat([data, evaluation_public], ignore_index=True)

#将预测集salesvolumn进行填充，方便进行滑窗
data.loc[data.regYear==2018, 'salesVolume'] = data[data.regYear==2018].forecastVolum.values   

#ema指标
@numba.jit
def get_EMA(df,a):
    for i in range(len(df)):
        if i<1320:
            df.ix[i,'ema']=df.ix[i,'salesVolume']
        if i>=1320:
            df.ix[i,'ema']=(1-a)*df.ix[i-1320,'ema'] + a*df.ix[i,'salesVolume']
    df.rename(columns={'ema': 'ema'+ '_' + str(round(a,3))}, inplace=True)
                
            

get_EMA(data,2/9)  #a = 1/(2N+1)   此处N取4

p_y_m = data.groupby(['province','regYear','regMonth'])['salesVolume'].mean().sort_index().reset_index()  #将groupby结果转为dataframe


data = pd.merge(data, p_y_m, on=['province', 'regYear', 'regMonth'], how='left', suffixes=('', '_p_y_m_mean'))

#popularity进行装箱
def get_bin(x):
    if x <=700:
        x = 400
    if (x>700) & (x<=1430):
        x = 1050
    if (x>1430) & (x<=2800):
        x = 2200
    if x>2800:
        x = 5000
    return x

data.popularity = data.popularity.apply(lambda x :get_bin(x))

#加入滞后特征
data['lag_4'] = np.array(lag_4)
data['lag_5'] = np.array(lag_5)
#data['lag_6'] = np.array(lag_6)

#将数据集进行列名转换并去除

data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['adcode']


#去除forecastVolum 缺失值，及去除2016 1-4 月数据
data = data[data.forecastVolum.notnull()]
#有滞后5阶，因而2016 5月数据也去除
data = data[data.lag_4!=0]
data = data[data.lag_5 !=0]

#提取特征季节
@numba.jit
def get_season(month):
    if 3<=month & month<6:
        season = 'spring'
    elif 6<=month & month<9:
        season = 'summer'
    elif 9<=month & month<12:
        season = 'autumn'
    else:
        season = 'winter'
    return season

month_value = data.regMonth.values
data['season'] = np.array([get_season(x) for x in month_value])
#df_data_season = pd.get_dummies(data.season, prefix = 'season_')
#data = pd.concat((data, df_data_season), axis=1)
#data.drop(['season'], axis=1, inplace=True)

#features要传入训练
num_feat = [ 'regMonth', 'regYear','popularity', 'carCommentVolum', 
            'newsReplyVolum','forecastVolum','lag_5','lag_4', 'ema_0.222',
           'salesVolume_p_y_m_mean']
cate_feat = [ 'bodyType', 'model', 'province', 'season']

for i in cate_feat:
    data[i] = data[i].astype('category')
features = num_feat + cate_feat


#将预测集salesvolumn重新变为0，好分割数据集
data.loc[data.regYear==2018, 'label'] = np.nan 

In [ ]:
def get_predict_w(model, data, label='label', feature=[], cate_feature=[], random_state=2018, n_splits=5,
                  model_type='lgb'):
    if 'sample_weight' not in data.keys():
        data['sample_weight'] = 1
    model.random_state = random_state
    predict_label = 'predict_' + label
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    data[predict_label] = 0
    test_index = (data[label].isnull()) | (data[label] == -1)    #找到要预测的数据集
    train_data = data[~test_index].reset_index(drop=True)     #分割出预测集训练集
    test_data = data[test_index]

    for train_idx, val_idx in kfold.split(train_data):
        model.random_state = model.random_state + 1

        train_x = train_data.loc[train_idx][feature]
        train_y = train_data.loc[train_idx][label]

        test_x = train_data.loc[val_idx][feature]
        test_y = train_data.loc[val_idx][label]
        if model_type == 'lgb':
            try:
                model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=400,
                          eval_metric='mae',
                          callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
                          categorical_feature=cate_feature,
                          sample_weight=train_data.loc[train_idx]['sample_weight'],
                          verbose=100)
            except:
                model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=200,
                          eval_metric='mae',
                          callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
                          categorical_feature=cate_feature,
                          sample_weight=train_data.loc[train_idx]['sample_weight'],
                          verbose=100)
        elif model_type == 'ctb':
            model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=200,
                      # eval_metric='mae',
                      # callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
                      cat_features=cate_feature,
                      sample_weight=train_data.loc[train_idx]['sample_weight'],
                      verbose=100)
        train_data.loc[val_idx, predict_label] = model.predict(test_x)
        if len(test_data) != 0:                  #预测集的预测
            test_data[predict_label] = test_data[predict_label] + model.predict(test_data[feature])
    test_data[predict_label] = test_data[predict_label] / n_splits
    print((train_data[label], train_data[predict_label]) * 5, train_data[predict_label].mean(),
          test_data[predict_label].mean())

    return pd.concat([train_data, test_data], sort=True, ignore_index=True), predict_label


lgb_model = lgb.LGBMRegressor(
    num_leaves=32, reg_alpha=0., reg_lambda=0.01, objective='mae', metric='rmse',
    max_depth=-1, learning_rate=0.05, min_child_samples=10, seed=2019,
    n_estimators=4000, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
)

data, predict_label = get_predict_w(lgb_model, data, label='label',
                                    feature=features, cate_feature=cate_feat,
                                    random_state=2019, n_splits=5)

data['lgb'] = data[predict_label]
data['forecastVolum'] = data['lgb'].apply(lambda x: -x if x < 0 else x)
df = data[data.label.isnull()][['id', 'forecastVolum']].round().astype(int)
df['forecastVolum'] = df['forecastVolum'].apply(lambda x: int(0.96 * x))
df.to_csv('./submit/rule.csv', index=False)

Training until validation scores don't improve for 400 rounds.
[100]	valid_0's rmse: 151.264	valid_0's l1: 75.367
[200]	valid_0's rmse: 146.403	valid_0's l1: 73.1058
[300]	valid_0's rmse: 144.209	valid_0's l1: 72.1831
[400]	valid_0's rmse: 143.658	valid_0's l1: 71.9413
[500]	valid_0's rmse: 143.49	valid_0's l1: 71.8565
[600]	valid_0's rmse: 143.303	valid_0's l1: 71.7842
[700]	valid_0's rmse: 143.138	valid_0's l1: 71.7064
[800]	valid_0's rmse: 142.965	valid_0's l1: 71.6316
[900]	valid_0's rmse: 142.842	valid_0's l1: 71.5655
[1000]	valid_0's rmse: 142.66	valid_0's l1: 71.4953
[1100]	valid_0's rmse: 142.537	valid_0's l1: 71.4377
[1200]	valid_0's rmse: 142.355	valid_0's l1: 71.3632
[1300]	valid_0's rmse: 142.209	valid_0's l1: 71.2915
[1400]	valid_0's rmse: 142.053	valid_0's l1: 71.2187
[1500]	valid_0's rmse: 141.939	valid_0's l1: 71.1663
[1600]	valid_0's rmse: 141.83	valid_0's l1: 71.1153
[1700]	valid_0's rmse: 141.698	valid_0's l1: 71.054
[1800]	valid_0's rmse: 141.609	valid_0's l1: 70.99

In [ ]:
df['forecastVolum'].mean()